# <span style="color:#F72585">Redes LSTM (Long Short Term Memory Networks)</span>

## <span style="color:#4361EE">Introducción</span>

Los humanos no comienzan a pensar desde cero cada segundo. A medida que usted lee este documento, comprende cada palabra en función de su comprensión de las palabras anteriores. No lee todo y empieza a pensar desde cero de nuevo. Sus pensamientos tienen persistencia.

Las redes neuronales tradicionales no pueden hacer esto, lo parece una gran deficiencia. Por ejemplo, imagine que desea clasificar qué tipo de evento está ocurriendo en cada punto de una película. No está claro cómo una red neuronal tradicional podría usar su razonamiento sobre eventos anteriores en la película para informar a los posteriores.

Las redes neuronales recurrentes (RNR) abordan este problema. Estas son redes con bucles que permiten que la información sea persistente.

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Redes_Recurrentes/Imagenes/RNN-rolled-sm.png" width="200" height="400" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Diagrama enrrollado de un trozo de una red neuronal recurrente</p>
</figcaption>
</figure>

Imagen tomada de [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)


El diagrama anterior presenta un diagrama *enrrollado* de un trozo de una red neuronal. La neurona $A$, observa alguna entrada $x_t$ y genera un valor $h_t$. Un bucle permite que la información trasite de un paso de la red al siguiente.

Estos bucles hacen que las redes neuronales recurrentes parezcan misteriosas. 


Sin embargo, si piensa un poco más, resulta que no son tan diferentes de una red neuronal normal. 

Una red neuronal recurrente puede considerarse como copias múltiples de la misma red, cada una de las cuales pasa un mensaje a un sucesor. Considere lo que sucede si desenrollamos el bucle:

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Redes_Recurrentes/Imagenes/RNN-unrolled.png" width="800" height="300" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Diagrama desenrrollado  de un trozo de una red neuronal recurrente </p>
</figcaption>
</figure>

Imagen tomada de [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)


Esta naturaleza en cadena revela que las redes neuronales recurrentes están íntimamente relacionadas con secuencias. 

Son la arquitectura natural de  red neuronal para usar con datos de naturaleza secuencial.

Desde el punto de vista estadístico son procesos estocásticos. Más precisamente, cadenas de Markov (ocultas).


En los últimos años, ha habido un éxito increíble aplicando RNR a una variedad de problemas: 

1. reconocimiento de voz,
2. modelamiento de idiomas, 
3. traducción, 
4. subtitulado de imágenes
5. ... La lista continúa. 


Dejaré la discusión de las increíbles hazañas que uno puede lograr con los RNN en la excelente publicación de blog de Andrej Karpathy, La irrazonable efectividad de las redes neuronales recurrentes http://karpathy.github.io/2015/05/21/rnn-effectiveness/ . Pero realmente son bastante asombrosos.


Esencial para estos éxitos es el uso de redes de tipo **LSTM**, un tipo muy especial de red neuronal recurrente que funciona, para muchas tareas, mucho mejor que la versión estándar. Casi todos los resultados emocionantes basados en redes neuronales recurrentes se logran con ellas. 

## <span style="color:#4361EE">El problema de las dependencias a largo plazo</span>

### <span style="color:#4CC9F0">Dependencias a corto plazo</span>


Uno de los atractivos de las RNR es la idea de que podrían ser capaces de conectar información previa a la tarea actual, como el uso de fotogramas de video anteriores que podrían informar la comprensión del presente cuadro. 


Si las RNR pudieran hacer esto, serían extremadamente útiles. ¿Pero pueden hacerlo? 

Depende.

A veces, solo necesitamos mirar información reciente para realizar la tarea actual. Por ejemplo, considere un modelo de lenguaje que intenta predecir la siguiente palabra en función de las anteriores. 


Si estamos tratando de predecir la última palabra en 

**las nubes están en el...**, 

no necesitamos más contexto; es bastante obvio que la siguiente palabra será 

**cielo**. 


En tales casos, en donde la brecha entre la información relevante y el lugar que se necesita es pequeña, las RNR pueden aprender a usar la información pasada.

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Redes_Recurrentes/Imagenes/RNN-shorttermdepdencies.png" width="800" height="300" align="center"/>
</center>
<figcaption>

RNR: dependencias de corto plazo. $h_3$ depende de $x_0, x_1,x_3,x_4$ 

</figcaption>
</figure>

Imagen tomada de [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Pero también hay casos en los que necesitamos más contexto. 

### <span style="color:#4CC9F0">Contexto a más largo plazo</span>


Consideremos el problema de  tratar de predecir la última palabra en el texto 

**"Crecí en Francia ... hablo francés con fluidez"**.


La información reciente sugiere que la siguiente palabra es probablemente el nombre de un idioma, pero si queremos limitar qué idioma, necesitamos el contexto de Francia, desde más atrás. 


Es completamente posible que la brecha entre la información relevante y el punto donde se necesita sea muy grande.

Desafortunadamente, a medida que crece esa brecha, los RNN no pueden aprender a conectar la información.

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Redes_Recurrentes/Imagenes/RNN-longtermdependencies.png" width="800" height="300" align="center"/>
</center>
<figcaption>
<p style="text-align:center">

RNR: dependencias de largo plazo. $h_{t+1}$ depende de $x_0, x_1$
 
</p>
</figcaption>
</figure>

Imagen tomada de [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Las redes neuronales luchan con dependencias a largo plazo. En teoría, los RNR son absolutamente capaces de manejar tales "dependencias a largo plazo". 


Un humano podría elegir cuidadosamente los parámetros para resolver problemas de juguetes de esta forma.

Lamentablemente, en la práctica, los RNR no parecen poder aprenderlos.

### <span style="color:#4CC9F0">El problema del gradiente que se desvanece o explota</span>



El problema fue explorado en profundidad por Hochreiter (1991) y Bengio, et al. (1994), quienes encontraron algunas razones fundamentales por las cuales podría ser difícil.

El problema fundamental mostrado por Hochreiter es que el **gradiente tiende a desaparecer o explotar** cuando en la RNR se calcula las transformaciones afines y se aplica la función de activación (el sigmoide) sobre las mismas matrices y vectores. Ver referencia 2 para más detalles.



¡Afortunadamente, los LSTM no tienen este problema!

## <span style="color:#4361EE">Redes LSTM </span> 



Las redes de memoria a corto y largo plazo, generalmente llamadas "LSTM", son un tipo especial de RNR, capaces de aprender
dependencias a largo plazo. 


Fueron introducidos por [Hochreiter y Schmidhuber (1997)](https://www.bioinf.jku.at/publications/older/2604.pdf), y fueron refinadas y popularizadas por muchas personas en trabajos posteriores. 

Funcionan tremendamente bien en una gran variedad de problemas, y ahora son ampliamente utilizadas.

Las redes LSTM están diseñadas explícitamente para resolver el problema de dependencia a largo plazo.


- **Recordar información durante largos períodos de tiempo es prácticamente su comportamiento predeterminado, ¡no es algo que les cuesta aprender!**

Todas las redes neuronales recurrentes tienen la forma de una cadena de módulos repetitivos de red neuronal. 


En los RNR estándar, este módulo repetitivo tendrá una estructura muy simple, como una sola capa de $\tanh$.

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Redes_Recurrentes/Imagenes/LSTM3-SimpleRNN.png" width="800" height="300" align="center"/>
</center>
<figcaption>
El módulo de repetición en una RNR estándar con una sola capa.
</figcaption>
</figure>

Imagen tomada de [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Las redes LSTM también tienen esta estructura tipo cadena, pero el módulo de repetición tiene una estructura diferente. En lugar de tener una sola capa de red neuronal, hay cuatro que interactúan de una manera muy especial.

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Redes_Recurrentes/Imagenes/LSTM3-chain.png" width="800" height="300" align="center"/>
</center>
<figcaption>
El módulo de repetición en una red LSTM  con cuatro capas interactuando.
</figcaption>
</figure>

Imagen tomada de [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)

No se preocupe por los detalles de lo que está sucediendo. Recorreremos el diagrama LSTM paso a paso más adelante. Por ahora, intentemos sentirnos cómodos con la notación que usaremos.

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Redes_Recurrentes/Imagenes/LSTM2-notation_1.png" width="800" height="300" align="center"/>
</center>
<figcaption>
Nomenclatura de los objetos en las redes LSTM 
</figcaption>
</figure>

Imagen tomada de [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)

En el diagrama anterior, cada línea transporta un vector completo, desde la salida de un nodo hasta las entradas de otros. Los círculos de color rosa representan operaciones puntuales, como la suma de vectores, mientras que los cuadros amarillos son capas de redes neuronales aprendidas. La fusión de líneas denota concatenación, mientras que una bifurcación de línea denota que su contenido se copia y las copias van a diferentes ubicaciones.

## <span style="color:#4361EE">La idea central detras de las redes LSTM</span> 

La clave para las redes LSTM es el estado de la celda, la línea horizontal que pasa por la parte superior del diagrama.

El estado de la celda es como una cinta transportadora. Corre directamente por toda la cadena, con solo algunas interacciones lineales menores. Es muy fácil que la información fluya sin cambios.

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Redes_Recurrentes/Imagenes/LSTM3-C-line.png" width="800" height="300" align="center"/>
</center>
<figcaption>
Banda transportadora  en las redes LSTM 
</figcaption>
</figure>

Imagen tomada de [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Una red LSTM tiene la capacidad de eliminar o agregar información al estado de la célula, cuidadosamente regulada por estructuras llamadas puertas.

Las puertas son una forma opcional de dejar pasar la información. Se componen de una capa de red neuronal con activación  sigmoide y una operación de multiplicación puntual.

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Redes_Recurrentes/Imagenes/LSTM3-gate.png" width="200" height="200" align="center"/>
</center>
<figcaption>
Puerta  en una red LSTM 
</figcaption>
</figure>

Imagen tomada de [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)

La capa sigmoide genera números entre cero y uno, que describe la cantidad de cada componente que debe dejarse pasar. 

- Un valor de cero significa **no dejar pasar nada**, 
- mientras que un valor de uno significa **dejar pasar todo**.

Un red LSTM tiene tres de estas puertas, para proteger y controlar el estado de la neurona.

## <span style="color:#4361EE"> Caminando a lo largo de una red LSTM. Paso a paso</span>

### <span style="color:#4CC9F0">Puerta de olvido</span>


El primer paso en nuestra red LSTM es decidir qué información vamos a tirar del estado de la celda. Esta decisión la toma una capa sigmoidea llamada **capa de puerta de olvido**. Se recibe $ h_{t − 1} $ y $ x_t $, y se genera un número entre 0 y 1 para cada número en el estado de celda $ C_{t − 1} $. Un 1 representa **mantener completamente esto** mientras que un 0 representa **deshacerse completamente de esto**.

Volvamos a nuestro ejemplo de un modelo de lenguaje que intenta predecir la siguiente palabra en función de todas las anteriores. 


En tal problema, el estado de la celda puede incluir el género del sujeto presente, de modo que se puedan usar los pronombres correctos. Cuando vemos un tema nuevo, queremos olvidar el género del tema anterior.

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Redes_Recurrentes/Imagenes/LSTM3-focus-f.png" width="800" height="300" align="center"/>
</center>
<figcaption>
Puerta  de olvido 
</figcaption>
</figure>

Imagen tomada de [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)

### <span style="color:#4CC9F0">Puerta de entrada</span>


El siguiente paso es decidir qué nueva información vamos a almacenar en el estado de la celda. Esto tiene dos partes. 

- Primero, una capa sigmoidea llamada **puerta de entrada** decide qué valores actualizaremos. 

- Luego, una capa de $\tanh$ crea un vector de nuevos valores candidatos, $\tilde{C}_t$, que podrían agregarse al estado. En el siguiente paso, combinaremos estos dos para crear una actualización del estado.

En el ejemplo de nuestro modelo de lenguaje, queremos agregar el género del nuevo sujeto al estado de la celda, para reemplazar el antiguo que estamos olvidando.

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Redes_Recurrentes/Imagenes/LSTM3-focus-i.png" width="800" height="600" align="center"/>
</center>
<figcaption>
Puerta  de entrada
</figcaption>
</figure>

Imagen tomada de [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)

### <span style="color:#4CC9F0">Actualización del estado de la celda</span>


Ahora es el momento de actualizar el estado de la celda anterior, $ C_{t − 1} $, en el nuevo estado de la celda $ C_t $. Los pasos anteriores ya decidieron qué hacer, solo tenemos que hacerlo.


Multiplicamos el estado anterior por $ f_t $, olvidando las cosas que decidimos olvidar antes. Luego agregamos $ i_t \times \tilde{C}_t $. Estos son los nuevos valores candidatos, escalados según cuánto decidimos actualizar cada valor de estado.

En el caso del modelo de lenguaje, aquí es donde realmente soltaríamos la información sobre el género del sujeto anterior y agreguemos la nueva información, como decidimos en los pasos anteriores.

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Redes_Recurrentes/Imagenes/LSTM3-focus-C.png" width="800" height="300" align="center"/>
</center>
<figcaption>
Actualización del estado de la celda
</figcaption>
</figure>

Imagen tomada de [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)

### <span style="color:#4CC9F0">Salida</span>


Finalmente, tenemos que decidir qué vamos a generar. Esta salida se basará en nuestro estado de celda, pero será una versión filtrada. Primero, ejecutamos una capa sigmoidea que decide qué partes del estado de la celda vamos a generar. Luego, ponemos el estado de la celda a través de tanh (para empujar los valores a estar entre -1 y 1) y lo multiplicamos por la salida de la puerta sigmoidea, de modo que solo produzcamos las partes que decidimos.

Para el ejemplo del modelo de lenguaje, dado que acaba de ver un tema, es posible que desee generar información relevante para un verbo, en caso de que eso sea lo que viene a continuación. Por ejemplo, podría generar si el sujeto es singular o plural, de modo que sepamos en qué forma se debe conjugar un verbo si eso es lo que sigue a continuación.

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Redes_Recurrentes/Imagenes/LSTM3-focus-o.png" width="800" height="300" align="center"/>
</center>
<figcaption>
Salida
</figcaption>
</figure>

Imagen tomada de [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)

## <span style="color:#4361EE">>Estructura Matemática de una red LSTM </span

En una red neuronal LSTM, cada puerta es una RNA. Una capa LSTM tiene varios bloques y cada bloque tiene dos salidas: $ C_t $, el estado actualizado de la celda y $ h_t $, el estado oculto actualizado.

$ C_t $ se actualiza de la siguiente manera

$$
\begin{align*}
f_t &= \sigma(W_f\cdot [h_{t-1}, x_t] + b_f)\\
i_t &= \sigma(W_i\cdot [h_{t-1}, x_t] + b_i)\\
\tilde{C}_t &= \tanh(W_c\cdot [h_{t-1}, x_t] + b_c)\\
C_t &= f_t \odot C_{t-1} + i_t\odot  \tilde{C}_t,\\
\end{align*}
$$

donde $ \odot $ es el producto de Hadamard o producto puntual de vectores.

Por otro lado, $ h_t $ se actualiza después de que $ C_t $ se haya actualizado de la siguiente manera

$$
\begin{align*}
o_t &= \sigma(W_o\cdot [h_{t-1}, x_t] + b_o)\\
h_t &=  o_t \odot \tanh(C_t).\\
\end{align*}
$$

Estas son las ecuaciones requeridas para calcular el gradiente de los parámetros de una capa LSTM.

## <span style="color:#4361EE">Cálculo del número de parámetros de una capa LSTM</span>

Supongamos que el tamaño de entrada es $n$ y que el tamaño de salida de la capa LSTM es *p*.

Entonces $h$ que es la salida tiene tamaño *p* y $x$ tiene tamaño *n*. Por lo tanto para que los operaciones indicadas puedan hacerse se requiere que

1. $W_f$, $W_c$, $W_i$ y $W_o$ tienen tamaño $p\times (p+n)$. Osea en total $4(p*p  + p*n)$.
2. Si se incluyen bias (que es lo común), son $4p$ parámetros adicionales, y en total se tiene que

$$
\text{Número de parámetros capa LSTM } = 4(p^2+ pn +p)
$$

La siguiente imagen muestra la estructura típica de una compuerta en el modelo LSTM. Se ilustra la activación *sigmoide* y *tanh*, porque téngase en cuenta que en la compuerta de actualización la activación es *tanh*.

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Redes_Recurrentes/Imagenes/compuerta_LSTM.jpg" width="800" height="600" align="center"/>
</center>
</figure>

Imagen: Alvaro Montenegro

Finalmente, la siguiente imagen revela el plano de una capa neuronal de tipo LSTM

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Redes_Recurrentes/Imagenes/plano_lstm.jpg" width="800" height="600" align="center"/>
</center>
</figure>

Imagen: Alvaro Montenegro

## <span style="color:#4361EE">Computación  en una capa LSTM de Keras</span>

Cualquier capa de Keras siempre espera un batch de datos. En el caso de una capa LSTM Keras espera tensores 3D de la siguiente forma

+ [batch_size, time_step, feature]



Por ejemplo un tensor de entrada de tamaño [32, 10, 8], la capa Keras lo interpreta como
- batch_size = 32, es decir 32 ejemplos.
- time_step = 10, es decir secuencias de entrada de tamaño 10. Por ejemplo en una serie de tiempo este es el tamaño de la ventana de entrada.
- feature = 8, es decir la variable de entrada es de tamaño 8. Por ejemplo, en series de timepo univariadas, feature = 1. En una serie multivariada con 8 variables, features = 8. En modelos de lenguaje natural feature = tamaño de representación de acada token. Usualmente correspondería al tamaño del embedding.

La salida de la capa corresponde al tamaño del estado oculto. Por ejemplo, si el estado oculto tiene tamaño, la salida de la capa es de tamaño [batch_size, 4].

### <span style="color:#4CC9F0">Ejemplo</span>


In [1]:
import tensorflow as tf
inputs = tf.random.normal([32,10,8])
lstm = tf.keras.layers.LSTM(4) # lstm es una capa de tamaño de salidad 4.
output = lstm(inputs)
print (output.shape)


(32, 4)


### <span style="color:#4CC9F0">Recibiendo toda la secuencia del valor del estado oculto</span>


En algunos casos es necesario disponer del valor del estado oculto para cada valor en la secuenci de entrada. Esta secuencia tiene tamaño [batch_size, time_step, output_size]
En el sugiente ejemplo se tiene que

- `return_sequences` son todos lo estados del estado oculto
- `return_state` es el último valor del estado oculto
- `final_carry_state` es el último valor de la banda transportadora

In [2]:
lstm = tf.keras.layers.LSTM(4, return_sequences = True, return_state=True)
whole_seq_output, final_memory_state, final_carry_state = lstm(inputs)

In [3]:
print(whole_seq_output.shape)
print(final_memory_state.shape)
print(final_carry_state.shape)

(32, 10, 4)
(32, 4)
(32, 4)


In [4]:
whole_seq_output[:,-1,:] == final_memory_state

<tf.Tensor: shape=(32, 4), dtype=bool, numpy=
array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True, 

### <span style="color:#4CC9F0">Extracción de los pesos de la capa</span>


In [5]:
help(lstm)

Help on LSTM in module keras.layers.rnn.lstm object:

class LSTM(keras.layers.rnn.dropout_rnn_cell_mixin.DropoutRNNCellMixin, keras.layers.rnn.base_rnn.RNN, keras.engine.base_layer.BaseRandomLayer)
 |  LSTM(units, activation='tanh', recurrent_activation='sigmoid', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0, return_sequences=False, return_state=False, go_backwards=False, stateful=False, time_major=False, unroll=False, **kwargs)
 |  
 |  Long Short-Term Memory layer - Hochreiter 1997.
 |  
 |  See [the Keras RNN API guide](https://www.tensorflow.org/guide/keras/rnn)
 |  for details about the usage of RNN API.
 |  
 |  Based on available runtime hardware and constraints, this layer
 |  wil

In [6]:
weights = lstm.get_weights()

In [7]:
len(weights)

3

In [8]:
weights[0].shape

(8, 16)

In [9]:
weights[1].shape

(4, 16)

In [10]:
weights[2].shape

(16,)

Note que los pesos de la capa LSTM Están organizados de la siguiente forma

In [11]:
units = 4 # tamaño del estado oculto
W = lstm.get_weights()[0]
W_x_f = W[:,:units]
W_x_i = W[:, units:units*2]
W_x_c = W[:, units*2:units*3]
W_x_o = W[:,units*3:]

U = lstm.get_weights()[1]
U_x_f = W[:,:units]
U_x_i = W[:, units:units*2]
U_x_c = W[:, units*2:units*3]
U_x_o = W[:,units*3:]

b = lstm.get_weights()[2] 
b_x_f = b[:units]
b_x_i = b[ units:units*2]
b_x_c = b[units*2:units*3]
b_x_o = b[units*3:]